In [ ]:
!pip install geopandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 69.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import geopandas as gpd
from scipy.spatial import cKDTree
from shapely.geometry import Point
from shapely.wkt import loads
import numpy as np

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the data
third_places_df = pd.read_excel('/content/drive/MyDrive/STUDIO_ML/ThirdPlacesByPostCode.xlsx')
# Define the paths to the .shp, .shx, and .dbf files
shp_path = '/content/drive/MyDrive/STUDIO_ML/POA_2021_AUST_GDA2020.shp'
postal_code_df = gpd.read_file(shp_path)

In [ ]:
# Filter postal_code_df to only include specific postal codes
postal_codes_to_include = ['3000','3002','3003','3004','3006','3008','3011','3012','3013','3015','3016','3018',
                           '3019','3020','3021','3022','3023','3024','3025','3026','3027','3028','3029','3030',
                           '3031','3032','3033','3034','3036','3037','3038','3039','3040','3041','3042','3043',
                           '3044','3045','3046','3047','3048','3049','3051','3052','3053','3054','3055','3056',
                           '3057','3058','3059','3060','3061','3062','3063','3064','3065','3066','3067','3068',
                           '3070','3071','3072','3073','3074','3075','3076','3078','3079','3081','3082','3083',
                           '3084','3085','3087','3088','3089','3090','3091','3093','3094','3095','3096','3097',
                           '3099','3101','3102','3103','3104','3105','3106','3107','3108','3109','3111','3113',
                           '3114','3115','3116','3121','3122','3123','3124','3125','3126','3127','3128','3129',
                           '3130','3131','3132','3133','3134','3135','3136','3137','3138','3140','3141','3142',
                           '3143','3144','3145','3146','3147','3148','3149','3150','3151','3152','3153','3154',
                           '3155','3156','3158','3159','3160','3161','3162','3163','3165','3166','3167','3168',
                           '3169','3170','3171','3172','3173','3174','3175','3177','3178','3179','3180','3181',
                           '3182','3183','3184','3185','3186','3187','3188','3189','3190','3191','3192','3193',
                           '3194','3195','3196','3197','3198','3199','3200','3201','3202','3204','3205','3206',
                           '3207','3211','3223','3335','3336','3337','3338','3427','3428','3429','3430','3431',
                           '3432','3433','3750','3751','3752','3753','3754','3755','3756','3757','3759','3761',
                           '3762','3763','3764','3765','3766','3770','3775','3781','3782','3783','3785','3786',
                           '3787','3788','3789','3791','3792','3793','3795','3796','3797','3799','3802','3803',
                           '3804','3805','3806','3807','3808','3809','3810','3910','3911','3912','3913','3915',
                           '3916','3918','3919','3920','3926','3927','3928','3929','3930','3931','3933','3934',
                           '3936','3937','3938','3939','3940','3941','3942','3943','3944','3975','3976','3977',
                           '3978','3980','3981']

postal_code_df = postal_code_df[postal_code_df['POA_CODE21'].isin(postal_codes_to_include)]


In [ ]:
print(postal_code_df.head())


    POA_CODE21 POA_NAME21 AUS_CODE21 AUS_NAME21  AREASQKM21  \
672       3000       3000        AUS  Australia      2.4814   
673       3002       3002        AUS  Australia      1.8557   
674       3003       3003        AUS  Australia      6.5822   
675       3004       3004        AUS  Australia      4.0421   
676       3006       3006        AUS  Australia      1.8542   

                                            LOCI_URI21  SHAPE_Leng  \
672  http://linked.data.gov.au/dataset/asgsed3/POA/...    0.083491   
673  http://linked.data.gov.au/dataset/asgsed3/POA/...    0.059488   
674  http://linked.data.gov.au/dataset/asgsed3/POA/...    0.139928   
675  http://linked.data.gov.au/dataset/asgsed3/POA/...    0.137947   
676  http://linked.data.gov.au/dataset/asgsed3/POA/...    0.066882   

     SHAPE_Area                                           geometry  
672    0.000254  POLYGON ((144.96139 -37.82050, 144.96063 -37.8...  
673    0.000190  POLYGON ((144.98978 -37.81906, 144.98973 -37.

In [ ]:
# Extract the x and y coordinates from third_places_df
third_places_coords = third_places_df[['latitude', 'longitude']].values

# Build the tree from the coordinates
tree = cKDTree(third_places_coords)


# Define a function to calculate the distances to the nearest 3 third places
def calculate_distances(row):
    # Initialize a Series of length 3 with np.inf
    distances = np.full(3, np.inf)
    # Get the centroid of the postal code area
    postal_code_centroid = row.geometry.centroid
    # Use the tree to get the distances to the nearest 3 third places
    dist, indices = tree.query([postal_code_centroid.x, postal_code_centroid.y], k=3)
    # Replace the values in distances with the actual distances as long as they exist
    distances[:len(dist)] = dist
    return pd.Series(distances)  # Now it returns a pandas Series

In [ ]:
print(postal_code_df)

     POA_CODE21 POA_NAME21 AUS_CODE21 AUS_NAME21  AREASQKM21  \
672        3000       3000        AUS  Australia      2.4814   
673        3002       3002        AUS  Australia      1.8557   
674        3003       3003        AUS  Australia      6.5822   
675        3004       3004        AUS  Australia      4.0421   
676        3006       3006        AUS  Australia      1.8542   
...         ...        ...        ...        ...         ...   
1352       3976       3976        AUS  Australia     13.3423   
1353       3977       3977        AUS  Australia    142.1782   
1354       3978       3978        AUS  Australia     96.5101   
1356       3980       3980        AUS  Australia     37.0278   
1357       3981       3981        AUS  Australia    203.7769   

                                             LOCI_URI21  SHAPE_Leng  \
672   http://linked.data.gov.au/dataset/asgsed3/POA/...    0.083491   
673   http://linked.data.gov.au/dataset/asgsed3/POA/...    0.059488   
674   http://linke

In [ ]:
# Apply the function to each row in postal_code_df and create a new column
postal_code_df[['dist_nearest', 'dist_2nd_nearest', 'dist_3rd_nearest']] = postal_code_df.apply(calculate_distances, axis=1)

# Now you can calculate the spatial dispersion index
postal_code_df['spatial_dispersion_index'] = postal_code_df[['dist_nearest', 'dist_2nd_nearest', 'dist_3rd_nearest']].std(axis=1)


In [ ]:
# Save the DataFrame as an Excel file
output_path = '/content/drive/MyDrive/STUDIO_ML/PostalCodeData.xlsx'
postal_code_df.to_excel(output_path, index=False)

In [ ]:
print(postal_code_df)

     POA_CODE21 POA_NAME21 AUS_CODE21 AUS_NAME21  AREASQKM21  \
672        3000       3000        AUS  Australia      2.4814   
673        3002       3002        AUS  Australia      1.8557   
674        3003       3003        AUS  Australia      6.5822   
675        3004       3004        AUS  Australia      4.0421   
676        3006       3006        AUS  Australia      1.8542   
...         ...        ...        ...        ...         ...   
1352       3976       3976        AUS  Australia     13.3423   
1353       3977       3977        AUS  Australia    142.1782   
1354       3978       3978        AUS  Australia     96.5101   
1356       3980       3980        AUS  Australia     37.0278   
1357       3981       3981        AUS  Australia    203.7769   

                                             LOCI_URI21  SHAPE_Leng  \
672   http://linked.data.gov.au/dataset/asgsed3/POA/...    0.083491   
673   http://linked.data.gov.au/dataset/asgsed3/POA/...    0.059488   
674   http://linke